In [1]:
from ipyfilechooser import FileChooser
import librosa
import IPython.display as ipd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

import sklearn
from scipy import stats
import numpy as np
import pandas as pd
from IPython.display import clear_output

import joblib

#Ottengo i classificatori
svm_clf = joblib.load("SVMClassifier.joblib")
lr_clf = joblib.load("LRClassifier.joblib")
knn_clf = joblib.load("KNNClassifier.joblib")
rf_clf = joblib.load("RFClassifier.joblib")
rf_definitions = joblib.load("RFDefinitions.joblib")

scaler = joblib.load("Scaler.joblib") 

# Create and display a FileChooser widget
fc = FileChooser()

# Sample callback function
def choose_song(chooser):
    clear_output(wait=True)
    display(fc)
    
    audio_length = librosa.get_duration(filename = fc.selected)
    x, sr = librosa.load(fc.selected, duration = 30, offset = (audio_length/2)-15)
    print('Duration: {:.2f}s, {} samples'.format(x.shape[-1] / sr, x.size))
    sr = 22050
    
    ipd.display(ipd.Audio(x, rate=sr))
    
    #################
    #Estrai info dall'audio
    mfccs = librosa.feature.mfcc(x, n_mfcc = 12, sr=sr)

    media_array, deviaz_array, max_array, min_array, mediana_array, skew_array, kurtosis_array = [], [], [], [], [], [], []
    for frequency_band in mfccs:
        media_array.append(np.mean(frequency_band))
        deviaz_array.append(np.std(frequency_band))
        max_array.append(np.amax(frequency_band))
        min_array.append(np.amin(frequency_band))
        mediana_array.append(np.median(frequency_band))
        skew_array.append(stats.skew(frequency_band))
        kurtosis_array.append(stats.kurtosis(frequency_band))
            
    mfcc_array = np.concatenate((media_array, deviaz_array, max_array, min_array, mediana_array, skew_array, kurtosis_array))
    
    
    chroma = librosa.feature.chroma_stft(x, sr=sr)
    media_array, deviaz_array, max_array, min_array, mediana_array, skew_array, kurtosis_array = [], [], [], [], [], [], []
    for pitch in chroma:
        media_array.append(np.mean(pitch))
        deviaz_array.append(np.std(pitch))
        max_array.append(np.amax(pitch))
        min_array.append(np.amin(pitch))
        mediana_array.append(np.median(pitch))
        skew_array.append(stats.skew(pitch))
        kurtosis_array.append(stats.kurtosis(pitch))

    chroma_array = np.concatenate((media_array, deviaz_array, max_array, min_array, mediana_array, skew_array, kurtosis_array))

    
    
    zero_crossing_rate = librosa.feature.zero_crossing_rate(x)

    media_zcr = np.mean(zero_crossing_rate)
    deviaz_zcr = np.std(zero_crossing_rate)
    massimo_zcr = np.amax(zero_crossing_rate)
    minimo_zcr = np.amin(zero_crossing_rate)
    mediana_zcr = np.median(zero_crossing_rate)

    zcr_array = np.array([media_zcr, deviaz_zcr, massimo_zcr, minimo_zcr, mediana_zcr])
    
    
    spec_centroid = librosa.feature.spectral_centroid(x, sr=sr)
    spec_rolloff = librosa.feature.spectral_rolloff(x, sr=sr)
    spec_bandwidth = librosa.feature.spectral_bandwidth(x, sr=sr)

    media_centroid = np.mean(spec_centroid)
    deviaz_centroid= np.std(spec_centroid)
    massimo_centroid = np.amax(spec_centroid)
    minimo_centroid = np.amin(spec_centroid)
    mediana_centroid = np.median(spec_centroid)

    media_rolloff = np.mean(spec_rolloff)
    deviaz_rolloff= np.std(spec_rolloff)
    massimo_rolloff = np.amax(spec_rolloff)
    minimo_rolloff = np.amin(spec_rolloff)
    mediana_rolloff = np.median(spec_rolloff)

    media_bandwidth = np.mean(spec_bandwidth)
    deviaz_bandwidth= np.std(spec_bandwidth)
    massimo_bandwidth = np.amax(spec_bandwidth)
    minimo_bandwidth = np.amin(spec_bandwidth)
    mediana_bandwidth = np.median(spec_bandwidth)


    centroid_array = np.array([media_centroid, deviaz_centroid, massimo_centroid, minimo_centroid, mediana_centroid])
    rolloff_array = np.array([media_rolloff, deviaz_rolloff, massimo_rolloff, minimo_rolloff, mediana_rolloff])
    bandwidth_array = np.array([media_bandwidth, deviaz_bandwidth, massimo_bandwidth, minimo_bandwidth, mediana_bandwidth])

    
    
    spec_contrasts = librosa.feature.spectral_contrast(x, sr=sr, n_bands = 6)        
    media_array, deviaz_array, max_array, min_array, mediana_array, skew_array, kurtosis_array = [], [], [], [], [], [], []
    for frequency_band in spec_contrasts[:-1]:
        media_array.append(np.mean(frequency_band))
        deviaz_array.append(np.std(frequency_band))
        max_array.append(np.amax(frequency_band))
        min_array.append(np.amin(frequency_band))
        mediana_array.append(np.median(frequency_band))
        skew_array.append(stats.skew(frequency_band))
        kurtosis_array.append(stats.kurtosis(frequency_band))

    contrast_array = np.concatenate((media_array, deviaz_array, max_array, min_array, mediana_array, skew_array, kurtosis_array))

    
    
    complete_array = np.concatenate((mfcc_array, chroma_array, zcr_array, centroid_array, rolloff_array, bandwidth_array, contrast_array))

    song_data = pd.DataFrame(np.reshape(complete_array, (1, -1)))

    scaled_song_data = scaler.transform(song_data)

    scaled_song_data_df = pd.DataFrame(np.reshape(scaled_song_data, (1, -1)))
    
    ipd.display(scaled_song_data_df)
    

    ###################

    svm_prediction = svm_clf.predict(scaled_song_data_df)
    print("Il classificatore SVM dice che la canzone è di genere: ",svm_prediction)
    
    lr_prediction = lr_clf.predict(scaled_song_data_df)
    print("Il classificatore Logistic Regression dice che la canzone è di genere: ",lr_prediction)
    
    knn_prediction = knn_clf.predict(scaled_song_data_df)
    print("Il classificatore KNN dice che la canzone è di genere: ",knn_prediction)
    
    rf_prediction = rf_clf.predict(scaled_song_data_df)
    #Reverse factorize
    reversefactor = dict(zip(range(8),rf_definitions))
    rf_prediction = np.vectorize(reversefactor.get)(rf_prediction)
    print("Il classificatore Random Forest dice che la canzone è di genere: ",rf_prediction)
    
# Register callback function
fc.register_callback(choose_song)

In [2]:
display(fc)

FileChooser(path='C:\Users\bonno\ProgettoBigData\PythonFiles\large dataset', filename='', title='HTML(value=''…

c:\users\bonno\anaconda3\envs\projectenv\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Duration: 30.00s, 661500 samples


,0,1,2,3,4,5,6,7,8,9,...,220,221,222,223,224,225,226,227,228,229
0,1.283721,-1.264211,-0.4378,1.069642,0.129787,0.411097,0.325975,0.111759,0.029381,0.237776,...,0.512947,0.835185,0.885513,-0.437877,2.396574,0.276717,-0.221379,0.119004,1.086655,-0.254481


Il classificatore SVM dice che la canzone è di genere:  ['Rock']
Il classificatore Logistic Regression dice che la canzone è di genere:  ['Rock']
Il classificatore KNN dice che la canzone è di genere:  ['Rock']
Il classificatore Random Forest dice che la canzone è di genere:  ['Rock']
